<a href="https://colab.research.google.com/github/lerlerchan/automation/blob/main/ExtractExcel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

ask pandas to read excel

In [3]:
# Read Excel File
df = pd.read_excel('supermarket_sales.xlsx')

In [4]:
# Select columns: 'Gender', 'Product line', 'Total'
df = df[['Gender', 'Product line', 'Total']]

In [5]:
#display df
df

,Gender,Product line,Total
0,Female,Health and beauty,548.9715
1,Female,Electronic accessories,80.2200
2,Male,Home and lifestyle,340.5255
3,Male,Health and beauty,489.0480
4,Male,Sports and travel,634.3785
...,...,...,...
995,Male,Health and beauty,42.3675
996,Female,Home and lifestyle,1022.4900
997,Male,Food and beverages,33.4320
998,Male,Home and lifestyle,69.1110


In [6]:
# Make pivot table
pivot_table = df.pivot_table(index='Gender', columns='Product line',
                             values='Total', aggfunc='sum').round(0)

In [7]:
# Export pivot table to Excel file
pivot_table.to_excel('pivot_table.xlsx', 'Report', startrow=4)

# **2. Add Chart**

In [8]:
from openpyxl import load_workbook
from openpyxl.chart import BarChart, Reference

In [9]:
# Read workbook and select sheet
wb = load_workbook('pivot_table.xlsx')
sheet = wb['Report']

In [10]:
# Active rows and columns
min_column = wb.active.min_column
max_column = wb.active.max_column
min_row = wb.active.min_row
max_row = wb.active.max_row

In [11]:
# Instantiate a barchart
barchart = BarChart()

In [12]:
# Locate data and categories
data = Reference(sheet,
                 min_col=min_column+1,
                 max_col=max_column,
                 min_row=min_row,
                 max_row=max_row)  # including headers

categories = Reference(sheet,
                       min_col=min_column,
                       max_col=min_column,
                       min_row=min_row+1,
                       max_row=max_row)  # not including headers

In [13]:
# Adding data and categories
barchart.add_data(data, titles_from_data=True)
barchart.set_categories(categories)

In [14]:
# Make chart
sheet.add_chart(barchart, "B12")
barchart.title = 'Sales by Product line'
barchart.style = 5  # choose the chart style

In [15]:
# Save workbook
wb.save('barchart.xlsx')

# **3. Apply formula**

In [16]:
from openpyxl.utils import get_column_letter

In [17]:
wb = load_workbook('barchart.xlsx')
sheet = wb['Report']

In [18]:
min_column = wb.active.min_column
max_column = wb.active.max_column
min_row = wb.active.min_row
max_row = wb.active.max_row

In [19]:
# Write multiple formulas with a for loop
for i in range(min_column+1, max_column+1):  # (B, G+1)
    letter = get_column_letter(i)
    sheet[f'{letter}{max_row + 1}'] = f'=SUM({letter}{min_row + 1}:{letter}{max_row})'
    sheet[f'{letter}{max_row + 1}'].style = 'Currency'

In [20]:
wb.save('report.xlsx')